# Building the dataset of research papers

The [Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module, a part of the [Biopython](http://biopython.org/) library, will be used to interface with [PubMed](http://www.ncbi.nlm.nih.gov/pubmed).<br>
You can download Biopython from [here](http://biopython.org/wiki/Download).

In this notebook we will be covering several of the steps taken in the [Biopython Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html), specifically in [Chapter 9  Accessing NCBI’s Entrez databases](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc109).

In [1]:
from Bio import Entrez

# NCBI requires you to set your email address to make use of NCBI's E-utilities
Entrez.email = "Your.Name.Here@example.org"

The datasets will be saved as serialized Python objects, compressed with bzip2.
Saving/loading them will therefore require the [pickle](http://docs.python.org/3/library/pickle.html) and [bz2](http://docs.python.org/3/library/bz2.html) modules.

In [2]:
import pickle, bz2, os

## EInfo: Obtaining information about the Entrez databases

In [3]:
# accessing extended information about the PubMed database
pubmed = Entrez.read( Entrez.einfo(db="pubmed"), validate=False )[u'DbInfo']

# list of possible search fields for use with ESearch:
search_fields = { f['Name']:f['Description'] for f in pubmed["FieldList"] }

In search_fields, we find 'TIAB' ('Free text associated with Abstract/Title') as a possible search field to use in searches.

In [4]:
search_fields

{'ALL': 'All terms from all searchable fields',
 'UID': 'Unique number assigned to publication',
 'FILT': 'Limits the records',
 'TITL': 'Words in title of publication',
 'WORD': 'Free text associated with publication',
 'MESH': 'Medical Subject Headings assigned to publication',
 'MAJR': 'MeSH terms of major importance to publication',
 'AUTH': 'Author(s) of publication',
 'JOUR': 'Journal abbreviation of publication',
 'AFFL': "Author's institutional affiliation and address",
 'ECNO': 'EC number for enzyme or CAS registry number',
 'SUBS': 'CAS chemical name or MEDLINE Substance Name',
 'PDAT': 'Date of publication',
 'EDAT': 'Date publication first accessible through Entrez',
 'VOL': 'Volume number of publication',
 'PAGE': 'Page number(s) of publication',
 'PTYP': 'Type of publication (e.g., review)',
 'LANG': 'Language of publication',
 'ISS': 'Issue number of publication',
 'SUBH': 'Additional specificity for MeSH term',
 'SI': 'Cross-reference from publication to other databases

## ESearch: Searching the Entrez databases

To have a look at the kind of data we get when searching the database, we'll perform a search for papers authored by Haasdijk:

In [5]:
example_authors = ['Haasdijk E']
example_search = Entrez.read( Entrez.esearch( db="pubmed", term=' AND '.join([a+'[AUTH]' for a in example_authors]) ) )
example_search

DictElement({'Count': '36', 'RetMax': '20', 'RetStart': '0', 'IdList': ['29311830', '28513205', '28513201', '28323435', '28140628', '26933487', '24977986', '24901702', '24852945', '24708899', '24252306', '23580075', '23144668', '22174697', '22154920', '21870131', '21760539', '20662596', '20602234', '20386726'], 'TranslationSet': [], 'TranslationStack': [DictElement({'Term': 'Haasdijk E[Author]', 'Field': 'Author', 'Count': '36', 'Explode': 'N'}, attributes={}), 'GROUP'], 'QueryTranslation': 'Haasdijk E[Author]'}, attributes={})

Note how the result being produced is not in Python's native string format:

In [6]:
type( example_search['IdList'][0] )

Bio.Entrez.Parser.StringElement

The part of the query's result we are most interested in is accessible through

In [7]:
example_ids = [ int(id) for id in example_search['IdList'] ]
print(example_ids)

[29311830, 28513205, 28513201, 28323435, 28140628, 26933487, 24977986, 24901702, 24852945, 24708899, 24252306, 23580075, 23144668, 22174697, 22154920, 21870131, 21760539, 20662596, 20602234, 20386726]


### PubMed IDs dataset

We will now assemble a dataset comprised of research articles containing the given keyword, in either their titles or abstracts.

In [56]:
search_term = 'allergy'

In [57]:
Ids_file = 'data/' + search_term + '_Ids.pkl.bz2'

In [58]:
if os.path.exists( Ids_file ):
    Ids = pickle.load( bz2.BZ2File( Ids_file, 'rb' ) )
else:
    # determine the number of hits for the search term
    search = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retmax=0 ) )
    total = int( search['Count'] )
    
    # `Ids` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Ids_str = []
    retrieve_per_query = 10000
    
    for start in range( 0, total, retrieve_per_query ):
        print('Fetching IDs of results [%d,%d]' % ( start, start+retrieve_per_query ) )
        s = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retstart=start, retmax=retrieve_per_query ) )
        Ids_str.extend( s[ u'IdList' ] )
    
    # convert Ids to integers (and ensure that the conversion is reversible)
    Ids = [ int(id) for id in Ids_str ]
    
    for (id_str, id_int) in zip(Ids_str, Ids):
        if str(id_int) != id_str:
            raise Exception('Conversion of PubMed ID %s from string to integer it not reversible.' % id_str )
    
    # Remove IDs that would cause problems below:
    Ids.remove(30823833)
    Ids.remove(30223658)
    Ids.remove(29623708)
    Ids.remove(29394076)
    Ids.remove(29303278)
    Ids.remove(28948806)
    Ids.remove(28832171)
    Ids.remove(28691830)
    Ids.remove(28264572)
    Ids.remove(28252306)
    Ids.remove(28078899)
    Ids.remove(27585420)
    
    # Save list of Ids
    pickle.dump( Ids, bz2.BZ2File( Ids_file, 'wb' ) )
    
total = len( Ids )
print('%d documents contain the search term "%s".' % ( total, search_term ) )

Fetching IDs of results [0,10000]
Fetching IDs of results [10000,20000]
Fetching IDs of results [20000,30000]
Fetching IDs of results [30000,40000]
Fetching IDs of results [40000,50000]
Fetching IDs of results [50000,60000]
Fetching IDs of results [60000,70000]
Fetching IDs of results [70000,80000]
73740 documents contain the search term "allergy".


Taking a look at what we just retrieved, here are the last 5 elements of the `Ids` list:

In [60]:
Ids[:5]

[31574307, 31573992, 31573731, 31573676, 31572454]

## ESummary: Retrieving summaries from primary IDs

To have a look at the kind of metadata we get from a call to `Entrez.esummary()`, we now fetch the summary of one of Haasdijk's papers (using one of the PubMed IDs we obtained in the previous section:

In [61]:
example_paper = Entrez.read( Entrez.esummary(db="pubmed", id='31574307') )[0]

def print_dict( p ):
    for k,v in p.items():
        print(k)
        print('\t', v)

print_dict(example_paper)

Item
	 []
Id
	 31574307
PubDate
	 2019 Sep 28
EPubDate
	 2019 Sep 28
Source
	 Ann Allergy Asthma Immunol
AuthorList
	 ['Harper V', 'Wagner A', 'Sowa P', 'Rutkowski K']
LastAuthor
	 Rutkowski K
Title
	 Allergy in frequent travelers: the alpha-gal story and the first UK case series.
Volume
	 
Issue
	 
Pages
	 
LangList
	 ['English']
NlmUniqueID
	 9503580
ISSN
	 1081-1206
ESSN
	 1534-4436
PubTypeList
	 ['Journal Article']
RecordStatus
	 PubMed - as supplied by publisher
PubStatus
	 aheadofprint
ArticleIds
	 DictElement({'pubmed': ['31574307'], 'medline': [], 'pii': 'S1081-1206(19)31239-6', 'doi': '10.1016/j.anai.2019.09.018', 'rid': '31574307', 'eid': '31574307'}, attributes={})
DOI
	 10.1016/j.anai.2019.09.018
History
	 DictElement({'pubmed': ['2019/10/02 06:00'], 'medline': ['2019/10/02 06:00'], 'received': '2019/07/17 00:00', 'revised': '2019/09/18 00:00', 'accepted': '2019/09/22 00:00', 'entrez': '2019/10/02 06:00'}, attributes={})
References
	 []
HasAbstract
	 0
PmcRefCount
	 0
FullJ

For now, we'll keep just some basic information for each paper: title, list of authors, publication year, and [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).

In case you are not familiar with the DOI system, know that the paper above can be accessed through the link  `https://www.doi.org/` followed by the paper's DOI.

In [62]:
( example_paper['Title'], example_paper['AuthorList'], int(example_paper['PubDate'][:4]), example_paper['DOI'] )

('Allergy in frequent travelers: the alpha-gal story and the first UK case series.',
 ['Harper V', 'Wagner A', 'Sowa P', 'Rutkowski K'],
 2019,
 '10.1016/j.anai.2019.09.018')

### Summaries dataset

We are now ready to assemble a dataset containing the summaries of all the paper `Ids` we previously fetched.

To reduce the memory footprint, and to ensure the saved datasets won't depend on Biopython being installed to be properly loaded, values returned by `Entrez.read()` will be converted to their corresponding native Python types. We start by defining a function for helping with the conversion of strings:

In [63]:
Summaries_file = 'data/' + search_term + '_Summaries.pkl.bz2'

In [64]:
if os.path.exists( Summaries_file ):
    Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
else:
    # `Summaries` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Summaries = []
    retrieve_per_query = 500
    
    print('Fetching Summaries of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        s = Entrez.read( Entrez.esummary( db="pubmed", id=query_ids ) )
        
        # out of the retrieved data, we will keep only a tuple (title, authors, year, DOI), associated with the paper's id.
        # (all values converted to native Python formats)
        for p in s:
            try:
                f = [
                    ( int( p['Id'] ), (
                        str( p['Title'] ),
                        [ str(a) for a in p['AuthorList'] ],
                        int( p['PubDate'][:4] ),                # keeps just the publication year
                        str( p.get('DOI', '') )            # papers for which no DOI is available get an empty string in their place
                        ) )
                    ]
                Summaries.extend( f )
            except ValueError as e:
                print("\nError with ID " + p['Id'] + ": " + str(e))
                print("Manually remove this ID above and re-run code.")
    
    # Save Summaries, as a dictionary indexed by Ids
    Summaries = dict( Summaries )
    
    pickle.dump( Summaries, bz2.BZ2File( Summaries_file, 'wb' ) )

Fetching Summaries of results: 

0...
Error with ID 30823833: invalid literal for int() with base 10: 'Wint'
Manually remove this ID above and re-run code.
...
Error with ID 30223658: invalid literal for int() with base 10: 'Summ'
Manually remove this ID above and re-run code.
...
Error with ID 29623708: invalid literal for int() with base 10: 'Spri'
Manually remove this ID above and re-run code.
.
Error with ID 29394076: invalid literal for int() with base 10: 'Fall'
Manually remove this ID above and re-run code.
.
Error with ID 29303278: invalid literal for int() with base 10: 'Wint'
Manually remove this ID above and re-run code.
.
Error with ID 28948806: invalid literal for int() with base 10: 'Fall'
Manually remove this ID above and re-run code.
.
Error with ID 28832171: invalid literal for int() with base 10: 'Summ'
Manually remove this ID above and re-run code.
.
Error with ID 28691830: invalid literal for int() with base 10: 'Summ'
Manually remove this ID above and re-run code.


Let us take a look at the first 3 retrieved summaries:

In [94]:
{ id : Summaries[id] for id in Ids[:3] }

{31574307: ('Allergy in frequent travelers: the alpha-gal story and the first UK case series.',
  ['Harper V', 'Wagner A', 'Sowa P', 'Rutkowski K'],
  2019,
  '10.1016/j.anai.2019.09.018'),
 31573992: ('Precision medicine in atopic diseases.',
  ['Cianferoni A', 'Annesi-Maesano I'],
  2019,
  '10.1097/ACI.0000000000000591'),
 31573731: ('Cinnamon extract inhibits allergen-specific immune responses in human and murine allergy models.',
  ['Ose R',
   'Tu J',
   'Schink A',
   'Maxeiner J',
   'Schuster P',
   'Lucas K',
   'Saloga J',
   'Bellinghausen I'],
  2019,
  '10.1111/cea.13507')}

## EFetch: Downloading full records from Entrez

`Entrez.efetch()` is the function that will allow us to obtain paper abstracts. Let us start by taking a look at the kind of data it returns when we query PubMed's database.

In [87]:
q = Entrez.read( Entrez.efetch(db="pubmed", id='18317433', retmode="xml") )['PubmedArticle']

`q` is a list, with each member corresponding to a queried id. Because here we only queried for one id, its results are then in `q[0]`.

In [88]:
type(q), len(q)

(list, 1)

1
At `q[0]` we find a dictionary containing two keys, the contents of which we print below.

In [89]:
type(q[0]), q[0].keys()

(Bio.Entrez.Parser.DictionaryElement,
 dict_keys(['MedlineCitation', 'PubmedData']))

In [90]:
print_dict( q[0][ 'PubmedData' ] )

ReferenceList
	 [DictElement({'Reference': [DictElement({'Citation': 'Diabetes. 2000 Apr;49(4):539-47', 'ArticleIdList': [StringElement('10871190', attributes={'IdType': 'pubmed'})]}, attributes={}), DictElement({'Citation': 'Nat Rev Immunol. 2002 Oct;2(10):748-59', 'ArticleIdList': [StringElement('12360213', attributes={'IdType': 'pubmed'})]}, attributes={}), DictElement({'Citation': 'Anticancer Drugs. 2007 Mar;18(3):237-44', 'ArticleIdList': [StringElement('17264754', attributes={'IdType': 'pubmed'})]}, attributes={}), DictElement({'Citation': 'J Biol Chem. 1999 Nov 5;274(45):32048-54', 'ArticleIdList': [StringElement('10542237', attributes={'IdType': 'pubmed'})]}, attributes={}), DictElement({'Citation': 'J Biol Chem. 2000 Oct 27;275(43):33201-4', 'ArticleIdList': [StringElement('10944516', attributes={'IdType': 'pubmed'})]}, attributes={}), DictElement({'Citation': 'J Biol Chem. 2005 Dec 23;280(51):42464-75', 'ArticleIdList': [StringElement('16227625', attributes={'IdType': 'pubmed

The key `'MedlineCitation'` maps into another dictionary. In that dictionary, most of the information is contained under the key `'Article'`. To minimize the clutter, below we show the contents of `'MedlineCitation'` excluding its `'Article'` member, and below that we then show the contents of `'Article'`.

In [91]:
print_dict( { k:v for k,v in q[0][ 'MedlineCitation' ].items() if k!='Article' } )

GeneralNote
	 []
KeywordList
	 []
SpaceFlightMission
	 []
CitationSubset
	 ['IM']
OtherID
	 []
OtherAbstract
	 []
PMID
	 18317433
DateCompleted
	 DictElement({'Year': '2008', 'Month': '04', 'Day': '24'}, attributes={})
DateRevised
	 DictElement({'Year': '2018', 'Month': '11', 'Day': '13'}, attributes={})
MedlineJournalInfo
	 DictElement({'Country': 'England', 'MedlineTA': 'J Investig Med', 'NlmUniqueID': '9501229', 'ISSNLinking': '1081-5589'}, attributes={})
ChemicalList
	 [DictElement({'RegistryNumber': '0', 'NameOfSubstance': StringElement('PPAR gamma', attributes={'UI': 'D047495'})}, attributes={})]
MeshHeadingList
	 [DictElement({'QualifierName': [], 'DescriptorName': StringElement('Humans', attributes={'UI': 'D006801', 'MajorTopicYN': 'N'})}, attributes={}), DictElement({'QualifierName': [StringElement('physiopathology', attributes={'UI': 'Q000503', 'MajorTopicYN': 'Y'})], 'DescriptorName': StringElement('Lung Diseases', attributes={'UI': 'D008171', 'MajorTopicYN': 'N'})}, attribu

In [92]:
print_dict( q[0][ 'MedlineCitation' ][ 'Article' ] )

ELocationID
	 [StringElement('10.2310/JIM.0b013e318165e89d', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})]
ArticleDate
	 []
Language
	 ['eng']
Journal
	 DictElement({'ISSN': StringElement('1081-5589', attributes={'IssnType': 'Print'}), 'JournalIssue': DictElement({'Volume': '56', 'Issue': '2', 'PubDate': DictElement({'Year': '2008', 'Month': 'Feb'}, attributes={})}, attributes={'CitedMedium': 'Print'}), 'Title': 'Journal of investigative medicine : the official publication of the American Federation for Clinical Research', 'ISOAbbreviation': 'J. Investig. Med.'}, attributes={})
ArticleTitle
	 PPARgamma: a novel molecular target in lung disease.
Pagination
	 DictElement({'MedlinePgn': '515-7'}, attributes={})
Abstract
	 DictElement({'AbstractText': ['Interest in peroxisome proliferator-activated receptors (PPARs) has steadily increased over the past 15 years. The recognition that subclasses of this receptor played critical roles in regulation of metabolism led to the development of sy

A paper's abstract can therefore be accessed with:

In [93]:
{ int(q[0]['MedlineCitation']['PMID']) : str(q[0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]) }

{18317433: 'Interest in peroxisome proliferator-activated receptors (PPARs) has steadily increased over the past 15 years. The recognition that subclasses of this receptor played critical roles in regulation of metabolism led to the development of synthetic ligands and their widespread application in the treatment of type 2 diabetes. At the same time, emerging evidence demonstrated that the influence of PPARs extends well beyond metabolism and diabetes. A salient example of this can be seen in studies that explore the role of PPARs in lung cell biology. In fact, current literature suggests that PPAR receptors may well represent exciting new targets for treatment in a variety of lung disorders. In an attempt to keep the scientific and medical communities abreast of these developments, a symposium sponsored by the American Federation for Medical Research entitled "PPARgamma: A Novel Molecular Target in Lung Disease" was convened on April 29, 2007, at the Experimental Biology Meeting in W

Some of the ids in our dataset refer to books from the [NCBI Bookshelf](http://www.ncbi.nlm.nih.gov/books/), a collection of freely available, downloadable, on-line versions of selected biomedical books. For such ids, `Entrez.efetch()` returns a slightly different structure, where the keys `[u'BookDocument', u'PubmedBookData']` take the place of the `[u'MedlineCitation', u'PubmedData']` keys we saw above.

### Abstracts dataset

We can now assemble a dataset mapping paper ids to their abstracts.

In [95]:
Abstracts_file = 'data/' + search_term + '_Abstracts.pkl.bz2'

In [96]:
import http.client
from collections import deque
from xml.dom import minidom
import re

def ch(node, childtype):
    return node.getElementsByTagName(childtype)[0]

if os.path.exists( Abstracts_file ):
    Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )
else:
    # `Abstracts` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Abstracts = deque()
    retrieve_per_query = 500
    
    print('Fetching Abstracts of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        # issue requests to the server, until we get the full amount of data we expect
        while True:
            try:
                #s = Entrez.read( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) )['PubmedArticle']
                s = minidom.parse( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) ).getElementsByTagName("PubmedArticle")
            except http.client.IncompleteRead:
                print('r', end='')
                continue
            break
        
        i = 0
        for p in s:
            abstr = ''
            if (p.getElementsByTagName('MedlineCitation')):
                citNode = ch(p,'MedlineCitation')
                pmid = ch(citNode,'PMID').firstChild.data
                articleNode = ch(citNode,'Article')
                if (articleNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            elif (p.getElementsByTagName('BookDocument')):
                bookNode = ch(p,'BookDocument')
                pmid = ch(bookNode,'PMID').firstChild.data
                if (bookNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            else:
                raise Exception('Unrecognized record type, for id %d (keys: %s)' % (Ids[start+i], str(p.keys())) )
            
            Abstracts.append( (int(pmid), str(abstr)) )
            i += 1
    
    # Save Abstracts, as a dictionary indexed by Ids
    Abstracts = dict( Abstracts )
    
    pickle.dump( Abstracts, bz2.BZ2File( Abstracts_file, 'wb' ) )

Fetching Abstracts of results: 

0...................
10000...................
20000...................
30000...................
40000...................
50000...................
60000...................
70000.......

Taking a look at one paper's abstract:

In [98]:
Abstracts[18317433]

'Interest in peroxisome proliferator-activated receptors (PPARs) has steadily increased over the past 15 years. The recognition that subclasses of this receptor played critical roles in regulation of metabolism led to the development of synthetic ligands and their widespread application in the treatment of type 2 diabetes. At the same time, emerging evidence demonstrated that the influence of PPARs extends well beyond metabolism and diabetes. A salient example of this can be seen in studies that explore the role of PPARs in lung cell biology. In fact, current literature suggests that PPAR receptors may well represent exciting new targets for treatment in a variety of lung disorders. In an attempt to keep the scientific and medical communities abreast of these developments, a symposium sponsored by the American Federation for Medical Research entitled "PPARgamma: A Novel Molecular Target in Lung Disease" was convened on April 29, 2007, at the Experimental Biology Meeting in Washington, 

## ELink: Searching for related items in NCBI Entrez

To understand how to obtain paper citations with Entrez, we will first assemble a small set of PubMed IDs, and then query for their citations.
To that end, we search here for papers published in the Nature journal with our given keyword.

In [99]:
CA_search_term = search_term+'[TIAB] AND Nature[JOUR]'
CA_ids = Entrez.read( Entrez.esearch( db="pubmed", term=CA_search_term ) )['IdList']
CA_ids

['24037376', '22538601', '22318520', '19587753', '17898760', '17581592', '17450126', '17377532', '16688147', '15496927', '15118685', '12490951', '12226655', '11449275', '11343123', '11130078', '10917520', '10688206', '10586891', '10586890']

In [100]:
CA_summ = {
    p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( CA_ids )) )
    }
CA_summ

{'24037376': ('Type 2 innate lymphoid cells control eosinophil homeostasis.',
  ['Nussbaum JC', 'Van Dyken SJ', 'von Moltke J', 'Cheng LE', 'Mohapatra A', 'Molofsky AB', 'Thornton EE', 'Krummel MF', 'Chawla A', 'Liang HE', 'Locksley RM'],
  '2013',
  'Nature',
  '10.1038/nature12526'),
 '22538601': ('Forum: Immunology: Allergy challenged.',
  ['Artis D', 'Maizels RM', 'Finkelman FD'],
  '2012',
  'Nature',
  '10.1038/484458a'),
 '22318520': ('Extrathymically generated regulatory T cells control mucosal TH2 inflammation.',
  ['Josefowicz SZ', 'Niec RE', 'Kim HY', 'Treuting P', 'Chinen T', 'Zheng Y', 'Umetsu DT', 'Rudensky AY'],
  '2012',
  'Nature',
  '10.1038/nature10772'),
 '19587753': ('Vision: New light on allergy receptor.',
  ['Grant M'],
  '2009',
  'Nature',
  '10.1038/460182a'),
 '17898760': ('Taking dendritic cells into medicine.',
  ['Steinman RM', 'Banchereau J'],
  '2007',
  'Nature',
  '10.1038/nature06175'),
 '17581592': ('CD1d-lipid-antigen recognition by the semi-invari

Because we restricted our search to papers in an open-access journal, you can then follow their DOIs to freely access their PDFs at the journal's website.

We will now issue calls to `Entrez.elink()` using these PubMed IDs, to retrieve the IDs of papers that cite them.
The database from which the IDs will be retrieved is [PubMed Central](http://www.ncbi.nlm.nih.gov/pmc/), a free digital database of full-text scientific literature in the biomedical and life sciences.

A complete list of the kinds of links you can retrieve with `Entrez.elink()` can be found [here](http://eutils.ncbi.nlm.nih.gov/entrez/query/static/entrezlinks.html).

In [101]:
CA_citing = {
    id : Entrez.read( Entrez.elink(
            cmd = "neighbor",               # ELink command mode: "neighbor", returns
                                            #     a set of UIDs in `db` linked to the input UIDs in `dbfrom`.
            dbfrom = "pubmed",              # Database containing the input UIDs: PubMed
            db = "pmc",                     # Database from which to retrieve UIDs: PubMed Central
            LinkName = "pubmed_pmc_refs",   # Name of the Entrez link to retrieve: "pubmed_pmc_refs", gets
                                            #     "Full-text articles in the PubMed Central Database that cite the current articles"
            from_uid = id                   # input UIDs
            ) )
    for id in CA_ids
    }

In [122]:
CA_citing['22538601']

[DictElement({'ERROR': [], 'LinkSetDb': [DictElement({'Link': [DictElement({'Id': '6033188'}, attributes={}), DictElement({'Id': '5525434'}, attributes={}), DictElement({'Id': '5010491'}, attributes={}), DictElement({'Id': '4790257'}, attributes={}), DictElement({'Id': '4729269'}, attributes={}), DictElement({'Id': '4715494'}, attributes={}), DictElement({'Id': '4593748'}, attributes={}), DictElement({'Id': '4547136'}, attributes={}), DictElement({'Id': '4265371'}, attributes={}), DictElement({'Id': '4164235'}, attributes={}), DictElement({'Id': '4026961'}, attributes={}), DictElement({'Id': '3924148'}, attributes={}), DictElement({'Id': '3789590'}, attributes={}), DictElement({'Id': '3459673'}, attributes={}), DictElement({'Id': '3434398'}, attributes={})], 'DbTo': 'pmc', 'LinkName': 'pubmed_pmc_refs'}, attributes={})], 'LinkSetDbHistory': [], 'DbFrom': 'pubmed', 'IdList': ['22538601']}, attributes={})]

We have in `CA_citing[paper_id][0]['LinkSetDb'][0]['Link']` the list of papers citing `paper_id`. To get it as just a list of ids, we can do

In [123]:
cits = [ l['Id'] for l in CA_citing['22538601'][0]['LinkSetDb'][0]['Link'] ]
cits

['6033188',
 '5525434',
 '5010491',
 '4790257',
 '4729269',
 '4715494',
 '4593748',
 '4547136',
 '4265371',
 '4164235',
 '4026961',
 '3924148',
 '3789590',
 '3459673',
 '3434398']

However, one more step is needed, as what we have now are PubMed Central IDs, and not PubMed IDs. Their conversion can be achieved through an additional call to `Entrez.elink()`:

In [124]:
cits_pm = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=",".join(cits)) )
cits_pm

[DictElement({'ERROR': [], 'LinkSetDb': [DictElement({'Link': [DictElement({'Id': '29944755'}, attributes={}), DictElement({'Id': '28790503'}, attributes={}), DictElement({'Id': '27225312'}, attributes={}), DictElement({'Id': '26776074'}, attributes={}), DictElement({'Id': '26410782'}, attributes={}), DictElement({'Id': '26299686'}, attributes={}), DictElement({'Id': '26228472'}, attributes={}), DictElement({'Id': '26210895'}, attributes={}), DictElement({'Id': '25525298'}, attributes={}), DictElement({'Id': '24657550'}, attributes={}), DictElement({'Id': '24592267'}, attributes={}), DictElement({'Id': '24210352'}, attributes={}), DictElement({'Id': '23827958'}, attributes={}), DictElement({'Id': '22970345'}, attributes={}), DictElement({'Id': '22886110'}, attributes={})], 'DbTo': 'pubmed', 'LinkName': 'pmc_pubmed'}, attributes={})], 'LinkSetDbHistory': [], 'DbFrom': 'pmc', 'IdList': ['6033188', '5525434', '5010491', '4790257', '4729269', '4715494', '4593748', '4547136', '4265371', '41

In [125]:
ids_map = { pmc_id : link['Id'] for (pmc_id,link) in zip(cits_pm[0]['IdList'], cits_pm[0]['LinkSetDb'][0]['Link']) }
ids_map

{'6033188': '29944755',
 '5525434': '28790503',
 '5010491': '27225312',
 '4790257': '26776074',
 '4729269': '26410782',
 '4715494': '26299686',
 '4593748': '26228472',
 '4547136': '26210895',
 '4265371': '25525298',
 '4164235': '24657550',
 '4026961': '24592267',
 '3924148': '24210352',
 '3789590': '23827958',
 '3459673': '22970345',
 '3434398': '22886110'}

And to check these papers:

In [126]:
{   p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( ids_map.values() )) )
    }

{'29944755': ('Immunoglobulin gene analysis as a tool for investigating human immune responses.',
  ['Dunn-Walters D', 'Townsend C', 'Sinclair E', 'Stewart A'],
  '2018',
  'Immunological reviews',
  '10.1111/imr.12659'),
 '28790503': ('Mast Cells and IgE can Enhance Survival During Innate and Acquired Host Responses to Venoms.',
  ['Galli SJ', 'Starkl P', 'Marichal T', 'Tsai M'],
  '2017',
  'Transactions of the American Clinical and Climatological Association',
  ''),
 '27225312': ('IgE and mast cells in host defense against parasites and venoms.',
  ['Mukai K', 'Tsai M', 'Starkl P', 'Marichal T', 'Galli SJ'],
  '2016',
  'Seminars in immunopathology',
  '10.1007/s00281-016-0565-1'),
 '26776074': ('The Mast Cell-IgE Paradox: From Homeostasis to Anaphylaxis.',
  ['Galli SJ'],
  '2016',
  'The American journal of pathology',
  '10.1016/j.ajpath.2015.07.025'),
 '26410782': ('IgE antibodies, FcεRIα, and IgE-mediated local anaphylaxis can limit snake venom toxicity.',
  ['Starkl P', 'Mari

### Citations dataset

We have now seen all the steps required to assemble a dataset of citations to each of the papers in our dataset.

In [127]:
Citations_file = 'data/' + search_term + '_Citations.pkl.bz2'
Citations = []

At least one server query will be issued per paper in `Ids`. Because NCBI allows for at most 3 queries per second (see [here](http://biopython.org/DIST/docs/api/Bio.Entrez-pysrc.html#_open)), this dataset will take a long time to assemble. Should you need to interrupt it for some reason, or the connection fail at some point, it is safe to just rerun the cell below until all data is collected.

In [139]:
import http.client

if Citations == [] and os.path.exists( Citations_file ):
    Citations = pickle.load( bz2.BZ2File( Citations_file, 'rb' ) )

if len(Citations) < len(Ids):
    
    i = len(Citations)
    checkpoint = int(len(Ids) / 100) + 1      # save to hard drive at every 1% of Ids fetched
    
    for pm_id in Ids[i:]:               # either starts from index 0, or resumes from where we previously left off
        
        while True:
            try:
                # query for papers archived in PubMed Central that cite the paper with PubMed ID `pm_id`
                c = Entrez.read( Entrez.elink( dbfrom = "pubmed", db="pmc", LinkName = "pubmed_pmc_refs", id=str(pm_id) ) )
                
                c = c[0]['LinkSetDb']
                if len(c) == 0:
                    # no citations found for the current paper
                    c = []
                else:
                    c = [ l['Id'] for l in c[0]['Link'] ]
                    
                    # convert citations from PubMed Central IDs to PubMed IDs
                    p = []
                    retrieve_per_query = 500
                    for start in range( 0, len(c), retrieve_per_query ):
                        query_ids = ','.join( c[start : start+retrieve_per_query] )
                        r = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=query_ids ) )
                        # select the IDs. If no matching PubMed ID was found, [] is returned instead
                        p.extend( [] if r[0]['LinkSetDb']==[] else [ int(link['Id']) for link in r[0]['LinkSetDb'][0]['Link'] ] )
                    c = p
            
            except http.client.BadStatusLine:
                # Presumably, the server closed the connection before sending a valid response. Retry until we have the data.
                print('r')
                continue
            except HTTPError:
                print('r')
                continue
            break
        
        Citations.append( (pm_id, c) )
        if (i % 10000 == 0):
            print('')
            print(i, end='')
        if (i % 100 == 0):
            print('.', end='')
        i += 1
        
        if i % checkpoint == 0:
            print('\tsaving at checkpoint', i)
            pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )
    
    print('\n done.')
    
    # Save Citations, as a dictionary indexed by Ids
    Citations = dict( Citations )
    
    pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )

....	saving at checkpoint 70848
.......	saving at checkpoint 71586
........	saving at checkpoint 72324
.......	saving at checkpoint 73062
.......
 done.


To see that we have indeed obtained the data we expected, you can match the ids below, with the ids listed at the end of last section.

In [140]:
Citations[22538601]

[29944755,
 28790503,
 27225312,
 26776074,
 26410782,
 26299686,
 26228472,
 26210895,
 25525298,
 24657550,
 24592267,
 24210352,
 23827958,
 22970345,
 22886110]

## Where do we go from here?

Running the code above generates multiple local files, containing the datasets we'll be working with. Loading them into memory is a matter of just issuing a call like<br>
``data = pickle.load( bz2.BZ2File( data_file, 'rb' ) )``.

The Entrez module will therefore no longer be needed, unless you wish to extend your data processing with additional information retrieved from PubMed.

Should you be interested in looking at alternative ways to handle the data, have a look at the [sqlite3](http://docs.python.org/3/library/sqlite3.html) module included in Python's standard library, or [Pandas](http://pandas.pydata.org/), the Python Data Analysis Library.